In [1]:
!pip install accelerate
!pip install peft bitsandbytes transformers trl sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00


In [2]:
import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#import bitsandbytes as bnb
import torch
import torch.nn as nn
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer
import transformers

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    )


In [3]:
model_name = "cyberagent/open-calm-large"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token   # When padding sequences to reach a fixed length, the tokenizer will not separate padding token. This means the padding itself will stop when it reaches the maximum token, to save memory.
tokenizer.padding_side = "right"  # add zeros to the right if the sentence reaches the end.

tokenizer_config.json:   0%|          | 0.00/326 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [5]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,     # It uses 4 bits format.  It will run faster but may be less accurate.
    bnb_4bit_quant_type="nf4",    # quantization type.
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,    # Use 4 bits for both weights and activations if set true.  Might be less accurate if set true.
    )

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map='auto' ,  #  {"": 0}  #'auto',   # {"": 0} set it to cpu.
    )

model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.78G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
prompt = "究極生命体カーズとは?"
#prompt = "あなたは誰"

#https://huggingface.co/docs/transformers/internal/generation_utils
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,
                max_length=100,
                temperature=0.9,
                top_p=0.75,
                top_k=40,
                repetition_penalty=4.0,  # default=5.0
                num_beams=2,  # default=10
                do_sample=True,
                )

for _ in range(5):
    result = pipe(f"{prompt}")
    print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


究極生命体カーズとは?
【画像】声優・麻倉ももさん、あまりにもエチエチな肉体を晒してしまう


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


究極生命体カーズとは?
【悲報】FF7リメイク、PS4でリマスター版が出るという噂が・・・
【朗報】ドラクエのラスボス「勇者」さん、とんでもないことになってしまうwwwwwww
【画像あり】ジャンプの新連載『ブラッククローバー』、ガチで面白そうwwwww
【衝撃】声優・悠木碧ちゃん、パンツ丸見え!
【速報】ワイ、スマブラに飽きる
【悲報】人気ユーチューバーさん、煽り運転


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


究極生命体カーズとは?
【悲報】FF7リメイク、発売まで後2週間なのに全然情報が無い件。
【朗報】PS4版『フォートナイト』のクロスプレイが正式実装!!
【画像あり】ジャンプ新連載「ワールドトリガー」の主人公、ガチで天才だったwww
【速報】新型Switchの発売日が決定!ついに日本上陸!!!
【悲報】本田翼さん、ゲーム実況生配信をするも配信中止の危機に直面する・・・



Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


究極生命体カーズとは?
【連載】自動車カタログで選ぶ厳選車! ニューモデル速報 小沢コージのクルマで行こう! 第1回「ホンダ・シビックタイプR」編(前編)
新車の見積もりや中古車購入を検討されている方は、ぜひ参考にしてみてください。
もっと読む
究極生命体カーズとは?
【悲報】声優・平野綾さん、安眠グッズに囲まれながらタバコを吸う!
『スマブラSP』発売3日で300万本販売!!「スマブラSP」は3700万台 「ジョイトイ」は2600万台突破wwwww
【朗報】アニメ『ゴールデンカムイ』、めちゃくちゃ面白そうwww
こち亀の川井ちゃん、あまりにも川井りおつすぎるwww
【画像】俺氏、ガチ


Load my pretrained model from hugging face.

In [8]:
my_model = "knkasa/open_calm_test"

peft_model = PeftModel.from_pretrained(model=model, model_id=my_model )

adapter_config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

In [9]:
prompt = "究極生命体カーズとは?"
#prompt = "あなたは誰"

#https://huggingface.co/docs/transformers/internal/generation_utils
pipe2 = pipeline(task="text-generation", model=peft_model, tokenizer=tokenizer,
                max_length=100,
                temperature=0.9,
                top_p=0.75,
                top_k=40,
                repetition_penalty=4.0,  # default=5.0
                num_beams=2,  # default=10
                do_sample=True,
                )

for _ in range(5):
    result = pipe2(f"{prompt}")
    print(result[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

究極生命体カーズとは?
- ディズニー・ピクサー映画「カーズ」に登場するキャラクターです。
- スティーブ・マックイーン主演の映画「カーズ」の主人公で、主人公の友人でもある。
- 自動車レースのチャンピオンである。
- ディズニーのアニメ映画「カーズ」に登場し、主人公のライバルとして登場する。
- マイケル・ベイ監督の映画「トランスフォーマー」にも登場します。
- スティーブン・スピルバーグ監督の映画「E.T.」にカメ


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


究極生命体カーズとは?
・X-MEN:アポカリプス
・デッドプール2
・アベンジャーズ/インフィニティ・ウォー
・スパイダーマン:ホームカミング
・マイティ・ソー バトルロイヤル
・ガーディアンズ・オブ・ギャラクシー リミックス
・アイアンマン3
・スター・ウォーズ エピソード1/ファントム・メナス
・ファンタスティック・ビーストと魔法使いの旅
・ドクター・ストレンジ
・キャプテン・マーベル
・アントマン
・


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


究極生命体カーズとは?
- レッド・ツェッペリン
- レディオヘッド
- ザ・ローリング・ストーンズ
- エレファント・カシノ
- パンク・ロック
- ブラック・サバス
- モトリー・クルーのメンバー
- マキシマムザホルモン
- ジミーハー・ブラザーズ
- アヴリル・ラヴィーン
- ビリー・ギボンズ
- デヴィッド・ボウイ
- ジャスティン・


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


究極生命体カーズとは?
2. カルマ・オブ・ザ・ゴッズ(The God of the Gods)
3. カーズ・オブ・テラー:Death and Dawn
4. ザ・カース・オブ・ゴッド(The Karms of Gods)
5. バットマン・イン・ザ・ボックス(Batman in the Boxes)
6. デビルズ・アンド・デス・オブ
究極生命体カーズとは?
カーズは、マーベル・コミックのヒーローである。彼は宇宙から地球へやって来て、地球に平和をもたらすために戦っている。しかし、彼の目的はただ一つ、人類を滅ぼすことにある。彼の名前は「キャプテン・アメリカ」で、その使命を果たすため、世界中の人々を犠牲にしてきたのだ。この物語は、スティーブ・ジョブズやマーク・ザッカーバーグなど、多くの有名人を生み出しています。また、多くの映画やテレビ番組にも登場しています。この映画は
